In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os

from PIL import Image, ImageDraw

In [12]:
pics = {}
for i in range(1,501):
    pics[i] = 'image_girl/0' + "{0:0=3d}".format(i) + '.jpg'

In [22]:
a = [[1,1,1,1],
     [2,2,2,2]]
b = [[3,3],
     [4,4],
     [5,5]]
np.asarray(a) * np.asarray(b)

ValueError: operands could not be broadcast together with shapes (2,4) (3,2) 

In [13]:
def exhaustive(next_image, window_size):
    """ EXHAUSTIVELY SEARCHES EACH WINDOW GIVEN A SIZE WITHIN AN IMAGE """
    temp0 = window_size[0]
    temp1 = window_size[1]
    window_size = (temp1, temp0)
    next_image = np.asarray(next_image)
    all_windows = []
    for y in range(0, next_image.shape[0]):
        for x in range(0, next_image.shape[1]):
            wind = next_image[y:y+window_size[1], x:x+window_size[0]]
            if wind.shape[0] == window_size[1] and wind.shape[1] == window_size[0]:
                all_windows.append((y,x,wind))
    return all_windows

# def ssd1(next_image, last_template):
#     # D = sum(u,v)(I(u,v) - T(u,v))^2
#     img_arr = np.asarray(next_image)
#     templ_arr = np.asarray(last_template)
#     shaped = templ_arr.shape
#     D = np.lib.stride_tricks.as_strided(img_arr,
#                                         shape=(img_arr.shape[0] - shaped[0] + 1,
#                                                img_arr.shape[1] - shaped[1] + 1) + shaped,
#                                         strides=img_arr.strides * 2)
#     ssd = np.einsum('ijkl,kl->ij', D, templ_arr)
#     ssd *= - 2
#     ssd += np.einsum('ijkl, ijkl->ij', D, D)
#     ssd += np.einsum('ij, ij', templ_arr, templ_arr)
#     return ssd

def SSD(next_image, last_template):
    """ CALCUALTE SSD BETWEEN WINDOW AND TEMPLATE """
    next_image = np.asarray(next_image)
    last_template = np.asarray(last_template)
    ssd = np.sum((next_image - last_template)**2)
    return ssd

def CC(next_image, last_template):
    """ CALCUALTE CC BETWEEN WINDOW AND TEMPLATE """
    next_image = np.asarray(next_image)
    last_template = np.asarray(last_template)
    cc = np.sum(next_image * last_template)
    return cc

def NCC(next_image, last_template):
    """ CALCUALTE NORMALIZED CC BETWEEN WINDOW AND TEMPLATE """
    next_image = np.asarray(next_image)
    last_template = np.asarray(last_template)
    
    I_hat = next_image - np.mean(next_image)
    T_hat = last_template - np.mean(last_template)
    numerator = np.sum(I_hat * T_hat)
    denominator = (np.sum(I_hat**2) * np.sum(T_hat**2))**.5
    ncc = numerator/denominator
    return ncc

In [14]:
def return_imgs(num, fn, coord_start=(19,52), box_size=(50,40), templ=None):
    """ SAVES AN IMAGE WITH A BOUNDARY BOX AROUND THE BEST MATCH
        TO THE GIVEN TEMPLATE IN AN IMAGE
    """
    im = Image.open(pics[num])
    name = fn.__name__
    
    # 50 tall, 40 wide
    yD = box_size[0]
    xD = box_size[1]
    ySt = coord_start[0]
    xSt = coord_start[1]
    if templ == None:
        templ = Image.fromarray(np.asarray(Image.open(pics[1]))[ySt:ySt+yD,xSt:xSt+xD,:])
    height,width = [np.asarray(templ).shape[i] for i in [0,1]]
    
    all_exhaustive = {}
    for each in exhaustive(im, (yD,xD)):
    #     all_exhaustive[each[0:2]] = 
        all_exhaustive[fn(each[-1], templ)] = each[0:2]
    #     all_exhaustive.append((each[0:2], ssd2(each[-1], templ)))
    # for i in sorted(all_exhaustive):
    #     print((i, all_exhaustive[i]))
    lowest_exhaustive = sorted(all_exhaustive)[0]
    highest_exhaustive = sorted(all_exhaustive)[-1]
    if name == 'SSD' or name == 'CC':
        start_coord = all_exhaustive[lowest_exhaustive]
    elif name == 'NCC':
        start_coord = all_exhaustive[highest_exhaustive]
    y,x = start_coord
    # lowest_exhaustive = np.where(total == sorted(total)[0])[0][0]

    # match = Image.fromarray(exhaustive(im1,(50,40))[lowest_exhaustive])
    # match

    top_l = (x,y)
    top_r = (x+xD,y)
    bot_r = (x+xD,y+yD)
    bot_l = (x,y+yD)
    ImageDraw.Draw(im).line([top_l, top_r, bot_r, bot_l, top_l], fill=(255,100,180), width=3)
    
    path = 'RESULTS/' + name + '_imgs/0' + "{0:0=3d}".format(num) + '.jpg'
    im = im.save(path)
    return im

In [15]:
def vid(folder_name):
    """ STITCHES IMAGES TOGETHER INTO A VIDEO """
    img_array = []
    for i in range(1,501):
        filename = 'RESULTS/' + folder_name + '_imgs/0' + "{0:0=3d}".format(i) + '.jpg'
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        img_array.append(img)
    out = cv2.VideoWriter('RESULTS/VIDEOS/' + folder_name + '.mp4', cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
    for i in range(len(img_array)):
        out.write(img_array[i])
    out.release()

In [21]:
# return_imgs() for SSD
for i in range(1,501):
    return_imgs(i, SSD)

# vid() for SSD
vid('SSD')

In [22]:
# return_imgs() for CC
for i in range(1,501):
    return_imgs(i, CC)

# vid() for CC
vid('CC')

In [23]:
# return_imgs() for NCC
for i in range(1,501):
    return_imgs(i, NCC)

# vid() for NCC
vid('NCC')